In [1]:
%run './template.ipynb'

In [2]:
import requests


API_KEY = config["API_KEY"]
BASE_URL = "https://trefle.io/api/v1/plants"


# for page in range(1, total_pages + 1):
params = {"token": API_KEY, "page": 10, "per_page": 20}
response = requests.get(BASE_URL, params=params)
response.json()

{'data': [{'id': 225947,
   'common_name': 'Meadow brome',
   'slug': 'bromus-erectus',
   'scientific_name': 'Bromus erectus',
   'year': 1762,
   'bibliography': 'Fl. Angl.: 89 (1762)',
   'author': 'Huds.',
   'status': 'accepted',
   'rank': 'species',
   'family_common_name': None,
   'genus_id': 10834,
   'image_url': 'https://bs.plantnet.org/image/o/ab6075a5082a2fb7d544819e19fb711988146045',
   'synonyms': ['Bromus erectus var. borbasii',
    'Bromopsis zangezura',
    'Bromopsis erecta subsp. gordjaginii',
    'Bromopsis caprina',
    'Bromus erectus var. versicolor',
    'Bromus erectus var. wildtii',
    'Bromus erectus var. subvillosus',
    'Zerna microchaeta',
    'Bromus caprinus',
    'Festuca fagetina',
    'Bromopsis permixta',
    'Bromus erectus var. glabriflorus',
    'Bromus erectus f. angustifolius',
    'Bromus erectus var. velutinus',
    'Bromus erectus var. piliferus',
    'Bromus erectus var. contractus',
    'Bromus erectus var. hirsutus',
    'Forasaccus er

In [ ]:
response.json()["data"][0]

{'id': 77116,
 'common_name': 'Evergreen oak',
 'slug': 'quercus-rotundifolia',
 'scientific_name': 'Quercus rotundifolia',
 'year': 1785,
 'bibliography': 'Encycl. 1: 723 (1785)',
 'author': 'Lam.',
 'status': 'accepted',
 'rank': 'species',
 'family_common_name': None,
 'genus_id': 3519,
 'image_url': 'https://d2seqvvyy3b8p2.cloudfront.net/40ab8e7cdddbe3e78a581b84efa4e893.jpg',
 'synonyms': ['Quercus ilex var. oleoides',
  'Quercus ilex subvar. rotundifolia',
  'Quercus ilex f. macrophylla',
  'Quercus ilex f. oleoides',
  'Quercus ilex var. calicina',
  'Quercus ilex subsp. rotundifolia',
  'Quercus lyauteyi',
  'Quercus ballota var. rotundifolia',
  'Quercus ilex f. brevicupulata',
  'Quercus ilex subvar. major',
  'Quercus ilex var. pendula',
  'Quercus rotundifolia f. dolichocalyx',
  'Quercus calicina',
  'Quercus rotundifolia f. pilosella',
  'Quercus rotundifolia f. macrocarpa',
  'Quercus ilex var. rotundifolia',
  'Quercus sugaro',
  'Quercus ilex subvar. pendula',
  'Quercu

In [3]:
from sqlmodel.ext.asyncio.session import AsyncSession
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker


engine = create_async_engine(sqlalchemy_url)
session_maker = async_sessionmaker(engine, expire_on_commit=False, class_=AsyncSession)

In [4]:
from sqlmodel import SQLModel, Field
from uuid import UUID, uuid4
from datetime import date


class Plant(SQLModel, table=True):
    __tablename__ = "plants"
    # __table_args__ = {"extend_existing": True}
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    origin_name: str
    name: str | None
    bibliography: str
    slug: str
    year: date
    image_url: str
    family: str
    genus: str
    rank: str

In [10]:
from datetime import date

async with session_maker() as session:
    for i in range(1, 6):
        params = {"token": API_KEY, "page": i, "per_page": 20}
        response = requests.get(BASE_URL, params=params)
        for plant in response.json()["data"]:
            session.add(
                Plant(
                    name=None,
                    origin_name=plant["common_name"],
                    bibliography=plant["bibliography"],
                    slug=plant["slug"],
                    year=date(year=plant["year"], month=1, day=1),
                    image_url=plant["image_url"],
                    family=plant["family"],
                    genus=plant["genus"],
                    rank=plant["rank"],
                )
            )

        await session.commit()

IntegrityError: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.NotNullViolationError'>: null value in column "origin_name" of relation "plants" violates not-null constraint
DETAIL:  Failing row contains (af87f513-f2d5-44d4-ba54-3a409f915f1e, null, Monogr. N. Amer. Potent. 2: 159 (1898), argentina-anserina, 1898-01-01, https://d2seqvvyy3b8p2.cloudfront.net/56e0e199c12748c3ab85be84bf..., Rosaceae, species, Argentina, null).
[SQL: INSERT INTO plants (id, origin_name, name, bibliography, slug, year, image_url, family, genus, rank) VALUES ($1::UUID, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR, $5::VARCHAR, $6::DATE, $7::VARCHAR, $8::VARCHAR, $9::VARCHAR, $10::VARCHAR)]
[parameters: [(UUID('d64fe32f-b821-4774-b4bb-e1e9a80c81b0'), 'Alpine strawberry', None, 'Sp. Pl.: 494 (1753)', 'fragaria-vesca', datetime.date(1753, 1, 1), 'https://bs.plantnet.org/image/o/4f45fd2d82661996f5d5a5613b39bdd1287a56bc', 'Rosaceae', 'Fragaria', 'species'), (UUID('54fb86d5-172c-4807-b99